In [3]:
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)

2023-06-20 11:32:30.274090: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-20 11:32:31.098304: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-20 11:32:31.104553: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 11:32:34.948326: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-20 11:32:42.194012: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2023-06-20 11:32:44.607806: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2023-06-20 11:

[[{'label': 'sadness', 'score': 0.0006792720523662865}, {'label': 'joy', 'score': 0.9959298968315125}, {'label': 'love', 'score': 0.0009452458471059799}, {'label': 'anger', 'score': 0.0018055305117741227}, {'label': 'fear', 'score': 0.0004111054295208305}, {'label': 'surprise', 'score': 0.00022885759244672954}]]


In [7]:
prediction[0]

[{'label': 'sadness', 'score': 0.0006792720523662865},
 {'label': 'joy', 'score': 0.9959298968315125},
 {'label': 'love', 'score': 0.0009452458471059799},
 {'label': 'anger', 'score': 0.0018055305117741227},
 {'label': 'fear', 'score': 0.0004111054295208305},
 {'label': 'surprise', 'score': 0.00022885759244672954}]

In [8]:
import os
from transformers import pipeline
import json
VIDEOS = ['jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp',
 'JFBB_Rhythm_Section_Day1_Morning Thomas',
 'JFBB_Rhythm_Section_Day1_Afternoon Thomas',
 'JFBB_Jazz_Funk_morning_video20230419101818 Luis Cal García',
 'JFBB_Jazz_Funk_afternoon_video20230419143359 Luis Cal García',
 'jazzaar-20230417-afternoon-JFLB-Cuban-Vocals-video Philipp',
 'jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp']

In [31]:
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

for video in VIDEOS:
    json_output = {'Video': video, 'predictions':[]}
    for root, dir, files in os.walk(f"../data/transcriptions/processed/{video}"):
        for file in files:
            
            with open(f'{root}/{file}') as f:
                text = f.read()
                f.close()
            prediction = classifier(text)

            segment_output = {'segment': file, 'emotions': prediction[0]}
            json_output.get('predictions').append(segment_output)
    
    sorted_data = sorted(json_output.get('predictions'), key = lambda x: int(x['segment'].split('_')[0]))
    json_output['predictions'] = sorted_data
    with open(f"../data/emotion_from_transcript/{video}.json", 'w') as file_path:
        json.dump(json_output, file_path)
    

Some layers from the model checkpoint at bhadresh-savani/distilbert-base-uncased-emotion were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/distilbert-base-uncased-emotion and are newly initialized: ['dropout_179']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/bsaut